In [1]:
# Add to environment variables PYTHONPATH = C:\Users\JFC_WIN\Documents\OpenSimula\src
import OpenSimula as osm
import pandas as pd

bestest_building_600 = {
    "name": "Bestest Building",
    "time_step": 3600,
    "n_time_steps": 8760,
    "initial_time": "01/01/2001 00:00:00",
    "components": [
        # MET_FILE
        {
            "type": "File_met",
            "name": "sevilla",
            "file_name": "met_files/sevilla.met"
        },
        # CONSTRUCTION
        {
            "type": "Material",
            "name": "Enlucido",
            "conductivity": 0.16,
            "density": 950,
            "specific_heat": 840
        },
        {
            "type": "Material",
            "name": "Aislamiento",
            "conductivity": 0.04,
            "density": 12,
            "specific_heat": 840
        },
        {
            "type": "Material",
            "name": "Madera pared",
            "conductivity": 0.14,
            "density": 530,
            "specific_heat": 900
        },
        {
            "type": "Material",
            "name": "Aislamiento suelo",
            "conductivity": 0.04,
            "density": .1,
            "specific_heat": .1,
        },
        {
            "type": "Material",
            "name": "Madera suelo",
            "conductivity": 0.14,
            "density": 650,
            "specific_heat": 1200,
        },
        {
            "type": "Material",
            "name": "Lecho_rocas",
            "conductivity": 0.14,
            "density": 650,
            "specific_heat": 1200,
        },
        {
            "type": "Construction",
            "name": "Pared 600",
            "solar_alpha": [0.6, 0.6],
            "materials": ["Enlucido", "Aislamiento", "Madera pared"],
            "thicknesses": [0.12, 0.066, 0.009],
        },
        {
            "type": "Construction",
            "name": "Suelo 600",
            "solar_alpha": [0.6, 0.6],
            "materials": ["Aislamiento suelo", "Madera suelo"],
            "thicknesses": [1.003, 0.025],
        },
        {
            "type": "Construction",
            "name": "Techo 600",
            "solar_alpha": [0.6, 0.6],
            "materials": ["Enlucido", "Aislamiento", "Madera pared"],
            "thicknesses": [0.010, 0.01118, 0.019],
        },
        {
            "type": "Window",
            "name": "Vidrio doble",
            "solar_alpha": [0.13, 0.13],
            "solar_transmisivity": [0.66, 0.66],
            "R_glazing": 0.921,
            "R_frame": 0.921,
            "frame_fraction": 0
        },
        # BUILDING
        {
            "type": "Building",
            "name": "Bestest 600",
            "file_met": "sevilla"
        },
        {
            "type": "Space_type",
            "name": "bestest space",
            "aux_variables": [],
            "people_density": "0",
            "light_density": "0",
            "other_gains_density": "4.1667",
            "other_gains_radiant_fraction": 0.6,
            "infiltration": "0.5"
        },
        {
            "type": "Space",
            "name": "space_1",
            "building": "Bestest 600",
            "space_type": "bestest space",
            "floor_area": 48,
            "volume": 48*2.7,
        },
        # Surfaces
        {
            "type": "Exterior_surface",
            "name": "north_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 8*2.7,
            "azimuth": 180,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "east_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 6*2.7,
            "azimuth": 90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 8*2.7,
            "azimuth": 0,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "west_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 6*2.7,
            "azimuth": -90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "roof",
            "construction": "Techo 600",
            "space": "space_1",
            "area": 8*6,
            "azimuth": 0,
            "altitude": 90
        },
        {
            "type": "Underground_surface",
            "name": "floor",
            "construction": "Suelo 600",
            "space": "space_1",
            "area": 8*6,
            "azimuth": 0,
            "altitude": -90
        },
    ],
}

In [2]:
sim = osm.Simulation()
pro = osm.Project("pro", sim)
pro.read_dict(bestest_building_600)
pro.simulate()

Reading project data from dictonary
Reading completed.
Checking project: Bestest Building
ok
Simulating Bestest Building: 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  End


In [3]:
pro.component("Bestest 600").KZ_matrix

array([[389.56720722]])

In [4]:
wall = "floor"
df_met = pro.component("sevilla").variable_dataframe()
df_space = pro.component("space_1").variable_dataframe()
df_wall = pro.component(wall).variable_dataframe()

df_graph = pd.DataFrame()
df_graph["date"] = df_met["date"]
df_graph["T_ext"] = df_met["temperature [°C]"]
df_graph["T_z"] = df_space["temperature [°C]"]
df_graph["T_s0"] = df_wall["T_s0 [°C]"]
df_graph["T_s1"] = df_wall["T_s1 [°C]"]

df_space.head(24)
#pro.component(wall).area

,date,temperature [°C],humidity [g/kg],people_convective [W],people_radiant [W],people_latent [W],light_convective [W],light_radiant [W],other_gains_convective [W],other_gains_radiant [W],other_gains_latent [W],solar_direct_gains [W],infiltration_flow [m³/s],surfaces_convective [W],delta_int_energy [W],infiltration_heat [W]
0,2001-01-01 00:00:00,17.917247,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.0,0.018,68.719029,96.571437,-245.291106
1,2001-01-01 01:00:00,16.088600,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.0,0.018,44.501016,84.789234,-209.290890
2,2001-01-01 02:00:00,16.174522,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.0,0.018,-52.016105,-3.983924,-24.000611
3,2001-01-01 03:00:00,16.109799,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.0,0.018,-49.514321,3.001029,-33.487347
4,2001-01-01 04:00:00,15.895443,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.0,0.018,-50.110504,9.939086,-39.829222
5,2001-01-01 05:00:00,15.584646,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.0,0.018,-48.395414,14.410762,-46.015988
6,2001-01-01 06:00:00,15.234413,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.0,0.018,-46.959830,16.239313,-49.280123
7,2001-01-01 07:00:00,14.859028,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.0,0.018,-45.410136,17.405563,-51.996067
8,2001-01-01 08:00:00,14.473119,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.0,0.018,-44.342306,17.893551,-53.551885
9,2001-01-01 09:00:00,14.244492,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.0,0.018,-49.617900,10.600783,-40.983523


In [5]:
import plotly.express as px

fig = px.line(df_graph,x='date',y=['T_ext','T_z','T_s0','T_s1'],height=400)
fig.show()